# Upload API folder to AWS S3 bucket

This is a temporary notebook for uploading all the files in the /api folder to a bucket on AWS S3

It's pretty inefficient as it currently stands. I've used the aws cli command to upload instead:

aws s3 sync ./api s3://'your-bucket-here' --profile 'your-profile-here'


In [4]:
# Imports and general variables

import sys
from boto3 import Session
from pathlib import Path
from tqdm import tqdm

root_path = Path(globals()['_dh'][0]).resolve().parent
sys.path.append(str(root_path))

import paths


In [2]:
# Config

session = Session(profile_name="600627346413_PowerUser_with_billing")
s3 = session.client("s3", region_name="eu-central-1")  # Frankfurt region

bucket_name = 'energy-toolkit-vgr-data'

directory = Path(paths.api_path)

all_files = [file_path for file_path in directory.rglob('*') if file_path.is_file()]


In [3]:
# Delete existing files in bucket

paginator = s3.get_paginator("list_objects_v2")
for page in paginator.paginate(Bucket=bucket_name):
    if "Contents" in page:
        delete_keys = {"Objects": [{"Key": obj["Key"]} for obj in page["Contents"]]}
        s3.delete_objects(Bucket=bucket_name, Delete=delete_keys)

In [3]:
with tqdm(total=len(all_files), desc="Uploading files", unit="file") as pbar:
    for file_path in all_files:
        key = file_path.relative_to(directory).as_posix()

        with file_path.open("rb") as data:
            s3.upload_fileobj(data, bucket_name, key)

        pbar.update(1)
        pbar.set_postfix(file=key)

Uploading files:   4%|▍         | 1136/28384 [02:35<1:00:18,  7.53file/s, file=geography=1460-1438-1439-1484-1461-1430-1421-1427-1486-1435-1488-1485-1487-1492,target-year=2030,self-sufficiency=0.925,energy-scenario=15000000,h2=False,offwind=False,biogas-limit=0.1/generators/biogas-market/power_t_1M.csv.gz]      